In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 18 14:33:13 2016

@author: pthouven
"""

%pylab inline
import os 
import pandas as pd
import numpy as np
import scipy.stats as sc
import sklearn
pd.set_option('display.max_columns', 400)
from sklearn.cross_validation import train_test_split


# import du csv
train=pd.read_csv('/home/foueslat/dtl-pnb/clean _script/data/encours_set/train_set_encours.csv',encoding='latin1',sep='|', index_col='contactid');
test=pd.read_csv('/home/foueslat/dtl-pnb/clean _script/data/encours_set/test_set_encours.csv',encoding='latin1',sep='|',index_col='contactid');
#tot=pd.read_csv('/home/foueslat/dtl-pnb/clean _script/data/SET_ENCOURS_AVEC_PARRAINAGE.csv',encoding='latin1',sep='|',index_col='contactid');
#INSEE=pd.read_csv('/data/home/pthouven/DataScience/dtl-pnb/revenu_commune.csv',sep=';');
#INSEE2=pd.DataFrame(INSEE, columns={'CODGEO','NBMENFISC12','NBPERSMENFISC12','MED12'})
#T1=train['PNB_PAR_CAPITA'].groupby(train['LABEL'])
#T2=T1.mean()
#print(T2)

Populating the interactive namespace from numpy and matplotlib


/opt/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# champs inutiles pour la modelisation
del train['Unnamed: 0']
del train['ENCOURS_CSL']
del train['ENCOURS_LVA']
del train['ENCOURS_CAV_CSL_LVA_LDD']
del train['ENCOURS_CAV']
del train['ENCOURS_LDD']
del train['ENCOURS_CAV_CSL_LVA_LDD_cat']

del test['Unnamed: 0']
del test['ENCOURS_CSL']
del test['ENCOURS_LVA']
del test['ENCOURS_CAV_CSL_LVA_LDD']
del test['ENCOURS_CAV']
del test['ENCOURS_LDD']
del test['ENCOURS_CAV_CSL_LVA_LDD_cat']

del train['phone_mobile']
del train['adresse_ligne_1']
del train['mail']
del train['namesponsor']
del train['ville']
del train['postalcodesponsor']
del train['code_postalf']
del train['paysf']

del test['phone_mobile']
del test['mail']
del test['adresse_ligne_1']
del test['namesponsor']
del test['postalcodesponsor']
del test['ville']
del test['code_postalf']
del test['paysf']

tot=pd.concat([train,test])
#del tot['code_postale_naissance']
tot.head()
tot['code_postale_naissance'] = tot2['code_postale_naissance'].astype(str)
tot['code_postale_naissance'].dtype

NameError: name 'tot2' is not defined

In [3]:
tot.shape

(27587, 20)

In [4]:
tot.dtypes

campagne                    int64
civilite                   object
flag_banque_principale    float64
code_postal                object
code_postale_naissance    float64
contibuableus             float64
csp                       float64
date_naissance             object
debit_cb                   object
nature_cb                  object
firstnamesponsor           object
flag_epargne               object
nom                        object
patrimoine                float64
pays                       object
pays_naissance_iso         object
regime_matrimonial        float64
revenus_annuels           float64
codeinsee                  object
LABEL                       int64
dtype: object

In [5]:
tot['LABEL'].describe()

count    27587.000000
mean         1.964802
std          1.449391
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: LABEL, dtype: float64

In [6]:
############################################## I- modelisation : datamanagement ######################################

def datamanagement(X,cible,chemin):
    
    #*************************** I- creation d un lieu de sauvgarde du datamanagement ******************
    import os
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing

    
    plus0="/"
    plus1=cible
    plus2="MOD"
    chemin1=chemin+plus0+plus2+plus0+plus1+plus0
    
    if os.path.isdir(chemin1)==True:
        os.chdir(chemin1)
    else:
        os.makedirs(chemin1) 
        os.chdir(chemin1)
    
 
    C = pd.DataFrame(X.columns, columns=['level_1'])
    C.to_csv("VARIABLE.csv",header=True, index=False,sep=';')
    
    #*************************** II- retaitement manuel de variables ******************
    
    from datetime import datetime
    X['civilite']=X['civilite'].replace(np.nan,'NR')
    X['code_postal']=X['code_postal'].replace('Vide','-2')
    X['code_postal']=X['code_postal'].replace(np.nan,'-2')


    now=datetime.now()
    X['AGE']=X['date_naissance'].replace(np.nan,'01/01/1800')
    X['AGE']= pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE']=X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']
    
    X['debit_cb']=X['debit_cb'].replace(np.nan,'SANS_CB')
    X['nature_cb']=X['nature_cb'].replace(np.nan,'SANS_CB')
    
    X['Parrain']=X['firstnamesponsor'].map(lambda x : 0 if x == np.nan else 1 )
    del X['firstnamesponsor']
    
    X['pays']=X['pays'].replace(np.nan,'NR')
    X['pays_naissance_iso']=X['pays_naissance_iso'].replace(np.nan,'NR')

    nb=-2
    X['regime_matrimonial']=X['regime_matrimonial'].fillna(nb)
    X['patrimoine']=X['patrimoine'].fillna(nb)
    X['revenus_annuels']=X['revenus_annuels'].fillna(nb)
    
    X[cible]=X['LABEL'].map(lambda x :1 if x==4 else 0)
  
    del X['LABEL']
    
    #*************************** III- transformation de donnees ******************
        
    #3.1. pour donnees quanti ou int on fait un cox box pour quali on regroup 
    columns1 = set(X.columns)
    columns1.remove(cible)
  
    import scipy.stats as sc
    for col in columns1:
        if X[col].dtype == np.dtype('float64') and len(np.unique(X[col].values))>=50:
            param=X[col].min()
            param2=pd.DataFrame({'valeur':[param]})
            param2.to_csv(col+"_param.csv",sep=';',encoding='utf-8')
            if param>=0:
                transformed_y, lambda_ = sc.boxcox(X[col]+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x+1))
                else:
                    X[col]=X[col].map(lambda x : ((x+1)**lambda_ -1)/lambda_)
            else:
                transformed_y, lambda_ = sc.boxcox(X[col]-param+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-param+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-param+1)**lambda_ -1)/lambda_)
                     
                
        if X[col].dtype == np.dtype('int64') and len(np.unique(X[col].values))>=50:
            param=X[col].min()
            param2=pd.DataFrame({'valeur':[param]})
            param2.to_csv(col+"_param.csv",sep=';',encoding='utf-8')
            if param>=0:
                transformed_y, lambda_ = sc.boxcox(X[col]+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x+1))
                else:
                    X[col]=X[col].map(lambda x : ((x+1)**lambda_ -1)/lambda_)
            else:
                transformed_y, lambda_ = sc.boxcox(X[col]-param+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-param+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-param+1)**lambda_ -1)/lambda_)
        
        if X[col].dtype == np.dtype('object') :
            if len(np.unique(X[col].values))<=20:
                regroup=pd.DataFrame(X[col].value_counts(normalize = True))
                regroup2=regroup[regroup[col]>=0.05].index
                X[col]=X[col].map(lambda x : x if x in regroup2 else 'AUTRE')
                le = preprocessing.LabelEncoder()
                G=pd.DataFrame(X[col])
                G['valeur']=le.fit_transform(G[col])
                L3=G.groupby(['valeur',col])[col].count()
                L3.to_csv(col+".csv",sep=';',encoding='utf-8')
                X[col]=le.fit_transform(X[col])
                
            else:
                
                regroup=pd.DataFrame(X[col].value_counts(normalize = True))
                if regroup[col].max()<=0.05:
                    le = preprocessing.LabelEncoder()
                    G=pd.DataFrame(X[col])
                    G['valeur']=le.fit_transform(G[col])
                    L3=G.groupby(['valeur',col])[col].count()
                    L3.to_csv(col+".csv",sep=';',encoding='utf-8')
                    X[col]=le.fit_transform(X[col])
                else:
                    regroup2=regroup[regroup[col]>=0.03].index
                    X[col]=X[col].map(lambda x : x if x in regroup2 else 'AUTRE')
                    le = preprocessing.LabelEncoder()
                    G=pd.DataFrame(X[col])
                    G['valeur']=le.fit_transform(G[col])
                    L3=G.groupby(['valeur',col])[col].count()
                    L3.to_csv(col+".csv",sep=';',encoding='utf-8')
                    X[col]=le.fit_transform(X[col])
                
    
    #3.2.Traitement des valeurs null     
    columns2 = set(X.columns)
    columns2.remove(cible)
    for col in columns2:
        if X[col].dtype == np.dtype('int64')and len(np.unique(X[col].values))>=50:
            median=X[col].median()
            export=pd.DataFrame({'valeur':[median]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(median)
        if X[col].dtype == np.dtype('int64')and len(np.unique(X[col].values))<50:
            mostfreq=pd.DataFrame(X[col].value_counts(normalize = True))
            mfreq=float(mostfreq.max())
            export=pd.DataFrame({'valeur':[mfreq]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(mfreq)
        if X[col].dtype == np.dtype('float64')and len(np.unique(X[col].values))>=50:
            median=X[col].median()
            export=pd.DataFrame({'valeur':[median]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(median)
        if X[col].dtype == np.dtype('float64')and len(np.unique(X[col].values))<50:
            mostfreq=pd.DataFrame(X[col].value_counts(normalize = True))
            mfreq=float(mostfreq.max())
            export=pd.DataFrame({'valeur':[mfreq]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(mfreq)
    
    par=list(X.columns)
    par.sort()
    X=pd.DataFrame(X,columns=par)
    to_drop=[cible]
    return X.drop(to_drop,axis=1),X[cible]
    



In [7]:
X,y=datamanagement(tot.copy(),'ENCOURS_6MOIS','/home/foueslat/dtl-pnb/Pilou_workspace/MOD/ENCOURS_6MOIS/')

/opt/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [8]:
tot.copy()['LABEL'].describe()

count    27587.000000
mean         1.964802
std          1.449391
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: LABEL, dtype: float64

In [9]:
y.describe()

count    27587.000000
mean         0.199587
std          0.399697
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: ENCOURS_6MOIS, dtype: float64

In [10]:

############################################# II- modelisation: construction du modele  ######################################
def modelisation (X,y,cible,chemin):  
    import numpy as np
    import pandas as pd
    import os
    u= cible
    v="proba"
    probabilite="%s_%s" %(u,v)
    
    #***********************************I-indicateurs de mesure de performence***********************
    
    def mesure(table,cible):
        import numpy as np
        import pandas as pd
        import os

        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[cible].groupby(decile['decile'])
        T2=pd.DataFrame(T1.mean())
        T2.rename(columns={probabilite:"taux_cible"},inplace=True)
        T2=T2.reset_index()
        result=(T2[T2['decile']==0][cible]/table[cible].mean())
        return result

    def proba_reco(table,cible):
        import numpy as np
        import pandas as pd
        import os

        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[probabilite].groupby(decile['decile'])
        T2=pd.DataFrame(T1.min())
        T2.rename(columns={probabilite:"proba_min_decile"},inplace=True)
        T2=T2.reset_index()
        proba=(T2[T2['decile']==0]["proba_min_decile"])
        return proba
    
    #***********************************II-modelisation ***************************************** 
    
    #1.0.echantillionnage
    from sklearn.cross_validation import train_test_split
    X0_train,X0_test,y0_train,y0_test=train_test_split(X,y,random_state=42)
    
    #1.1. modules necessaire
    
    from sklearn import linear_model,decomposition
    from sklearn.pipeline import Pipeline
    from sklearn.feature_selection import RFE,SelectFromModel
    from sklearn import metrics
    from sklearn import  grid_search
    from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier
    from sklearn.calibration import CalibratedClassifierCV
    from sklearn import svm
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    #1.2. modele 1
    try:
        parameters = {'n_estimators':[150,300,350], 'n_jobs':[-1]}
        rfc= RandomForestClassifier()
        m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        m0 = m0.fit(X0_train,y0_train)
        proba0=pd.DataFrame(y0_test)
        P = m0.predict_proba(X0_test)[:, 1]
        proba0[probabilite]= P
        R0=mesure(proba0,cible)
        R0=pd.DataFrame(R0)
        R0.rename(columns={cible:"R0"},inplace=True)
        print(R0)
        
    except ValueError:
        R0=pd.DataFrame([0],index=[9],columns=["R0"])
        print(R0)

    #1.3. modele 2
    try:
        parameters = {'n_estimators':[300,400], 'n_jobs':[-1]}
        rfc= ExtraTreesClassifier()
        m1=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        m1 = m1.fit(X0_train,y0_train)
        proba1=pd.DataFrame(y0_test)
        P = m1.predict_proba(X0_test)[:, 1]
        proba1[probabilite]= P
        R1=mesure(proba1,cible)
        R1=pd.DataFrame(R1)
        R1.rename(columns={cible:"R1"},inplace=True)
        print(R1)
        
    except ValueError:
        R1=pd.DataFrame([0],index=[9],columns=["R1"])
        print(R1)
    
    #1.4. modele 3
    try:
        parameters = {'n_estimators':[150,300,350]}
        rfc= GradientBoostingClassifier(max_depth=7)
        m2=grid_search.GridSearchCV(rfc,parameters,cv=6)
        m2 = m2.fit(X0_train,y0_train)
        P=m2.predict(X0_test)
        proba2=pd.DataFrame(y0_test)
        P = m2.predict_proba(X0_test)[:, 1]
        proba2[probabilite]= P
        R2=mesure(proba2,cible)
        R2=pd.DataFrame(R2)
        R2.rename(columns={cible:"R2"},inplace=True)
        print(R2)
    
        
    except ValueError:
        R2=pd.DataFrame([0],index=[9],columns=["R2"])
        print(R2)
    
    #1.5.modele 4
    try:
        rfc= RandomForestClassifier(n_estimators=200, n_jobs=-1)
        logistic = linear_model.LogisticRegression(solver='liblinear',C=100,n_jobs=-1)
        select=SelectFromModel(logistic,threshold="median")
        a0 = CalibratedClassifierCV(rfc, cv=6, method='isotonic')
        m3 = Pipeline(steps=[('feature_selection', select),('regression', a0)])
        m3 = m3.fit(X0_train,y0_train)
        P = m3.predict_proba(X0_test)[:, 1]
        proba3=pd.DataFrame(y0_test)
        proba3[probabilite]= P
        R3=mesure(proba3,cible)
        R3=pd.DataFrame(R3)
        R3.rename(columns={cible:"R3"},inplace=True)
        print(R3)
    
    except ValueError:
        R3=pd.DataFrame([0],index=[9],columns=["R3"])
        print(R3)
        
    #1.6.modele 5
    try:
        rfc= GradientBoostingClassifier(n_estimators=200,max_depth=8)
        logistic = linear_model.LogisticRegression(solver='liblinear',C=100,n_jobs=-1)
        select=SelectFromModel(logistic,threshold="median")
        a0 = CalibratedClassifierCV(rfc, cv=6, method='sigmoid')
        m4 = Pipeline(steps=[('feature_selection', select),('regression', a0)])
        m4 = m4.fit(X0_train,y0_train)
        P = m4.predict_proba(X0_test)[:, 1]
        proba4=pd.DataFrame(y0_test)
        proba4[probabilite]= P
        R4=mesure(proba4,cible)
        R4=pd.DataFrame(R4)
        R4.rename(columns={cible:"R4"},inplace=True)
        print(R4)
    
    except ValueError:
        R4=pd.DataFrame([0],index=[9],columns=["R4"])
        print(R4)
    
    #1.7.modele 6
    try:
        etc= ExtraTreesClassifier(n_estimators=400,max_depth=10,n_jobs=-1)
        m5=etc.fit(X0_train,y0_train)
        P = m5.predict_proba(X0_test)[:, 1]
        proba5=pd.DataFrame(y0_test)
        proba5[probabilite]= P
        R5=mesure(proba5,cible)
        R5=pd.DataFrame(R5)
        R5.rename(columns={cible:"R5"},inplace=True)
        print(R5)
    
    except ValueError:
        R5=pd.DataFrame([0],index=[9],columns=["R5"])
        print(R5)
    
    #1.8.modele 7
    try:
        parameters = {'n_estimators':[200,300], 'max_depth':[10,18],'n_jobs':[-1]}
        rfc= RandomForestClassifier()
        m6=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1).fit(X0_train,y0_train)
        proba6=pd.DataFrame(y0_test)
        P = m6.predict_proba(X0_test)[:, 1]
        proba6[probabilite]= P
        R6=mesure(proba0,cible)
        R6=pd.DataFrame(R6)
        R6.rename(columns={cible:"R6"},inplace=True)
        print(R6)
        
    except ValueError:
        R6=pd.DataFrame([0],index=[9],columns=["R6"])
        print(R6)
       
    try:
        coef=[0.14,0.14,0.14,0.14,0.14,0.14,0.14]
        proba7=coef[0]*proba0+coef[1]*proba1+coef[2]*proba2+coef[3]*proba3+coef[4]*proba4+coef[5]*proba5+coef[6]*proba6
        R7=mesure(proba7,cible)
        R7=pd.DataFrame(R7)
        R7.rename(columns={cible:"R7"},inplace=True)
        coef2=coef
        coef=pd.DataFrame(coef2,columns=['valeur'])
        coef.to_csv("coefmodel.csv",sep=';',encoding='utf-8')
        print(R7)
    except ValueError:
        R7=pd.DataFrame([0],index=[9],columns=["R7"])
        print(R7)
    
    try:
        R9=R7
        R9.rename(columns={"R7":"R8"},inplace=True)
        
        for i in xrange(1,100000):
            li=list(np.random.random(7))
            li=li/sum(li)
            coef=list(li)
            proba8=coef[0]*proba0+coef[1]*proba1+coef[2]*proba2+coef[3]*proba3+coef[4]*proba4+coef[5]*proba5+coef[6]*proba6
            R8=mesure(proba8,cible)
            R8=pd.DataFrame(R8)
            R8.rename(columns={cible:"R8"},inplace=True)
            if float(R8["R8"])>float(R9["R8"]):
                R9=R8
                coef2=coef
        coef=pd.DataFrame(coef2,columns=['valeur'])
        coef.to_csv("coefmodel.csv",sep=';',encoding='utf-8')
        print(R9)
    except ValueError:
        R8=pd.DataFrame([0],index=[9],columns=["R8"])
        print(R8)
        
     
    #***********************************III-choix du modele *****************************************
    import cPickle
    plus0="/"
    plus1=cible
    plus2="MOD"
    chemin1=chemin+plus2+plus0+plus1+plus0
    
    if os.path.isdir(chemin1)==True:
        os.chdir(chemin1)
    else:
        os.makedirs(chemin1) 
        os.chdir(chemin1)

        
    R=pd.concat([R0,R1,R2,R3,R4,R5,R6,R7],axis=1)
    RR=R.T
    RR=pd.DataFrame(RR)
    RR.rename(columns=lambda x: 'valeur',inplace=True)
    RR.sort_values(by='valeur',ascending=False,inplace=True)
    RRR=pd.DataFrame(RR.head(1))
    param=RRR.index

     
    if param=="R0":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m0,f)
        export=proba_reco(proba0,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')   
    if param=="R1":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m1,f)
        export=proba_reco(proba1,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
    if param=="R2":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m2,f)
        export=proba_reco(proba2,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
        
    if param=="R3":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m3,f)
        export=proba_reco(proba3,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
        
    if param=="R4":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m4,f)
        export=proba_reco(proba4,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R5":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m5,f)
        export=proba_reco(proba5,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R6":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m6,f)
        export=proba_reco(proba6,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R7":
        with open(chemin1+"M0.pickle",'wb') as f:
            cPickle.dump(m0,f)
        with open(chemin1+"M1.pickle",'wb') as f:
            cPickle.dump(m1,f)  
        with open(chemin1+"M2.pickle",'wb') as f:
            cPickle.dump(m2,f)
        with open(chemin1+"M3.pickle",'wb') as f:
            cPickle.dump(m3,f)
        with open(chemin1+"M4.pickle",'wb') as f:
            cPickle.dump(m4,f)
        with open(chemin1+"M5.pickle",'wb') as f:
            cPickle.dump(m5,f)
        with open(chemin1+"M6.pickle",'wb') as f:
            cPickle.dump(m6,f)
        export=proba_reco(proba7,cible)
        export.to_csv(cible+"reco_proba.csv",sep=';',encoding='utf-8')
        
    
    print(param)

    
####################################### III-Application modele ######################################################

def application(X,cible,cheminmodel):
    
    
    #*************************** I- chargement des donnees pour datamanagement ******************
    import os
    import cPickle
    import numpy as np
    import pandas as pd
    
    u= cible
    v="proba"
    probabilite="%s_%s" %(u,v)
    
    
    if os.path.isdir(cheminmodel)==True:
        os.chdir(cheminmodel)
    else:
        os.makedirs(cheminmodel)
        os.chdir(cheminmodel)
    

 
    var=pd.read_csv("VARIABLE.csv",sep=';',encoding="latin-1")['level_1'].values
    var2=list(var)
    X=pd.DataFrame(X, columns=var2)
    X.rename(columns={'LABEL':cible},inplace=True)

    
    
    #*************************** II- retaitement manuel de variables ******************
    
     
    from datetime import datetime
    X['civilite']=X['civilite'].replace(np.nan,'NR')
    X['code_postal']=X['code_postal'].replace('Vide','-2')
    X['code_postal']=X['code_postal'].replace(np.nan,'-2')


    now=datetime.now()
    X['AGE']=X['date_naissance'].replace(np.nan,'01/01/1800')
    X['AGE']= pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE']=X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']
    
    X['debit_cb']=X['debit_cb'].replace(np.nan,'SANS_CB')
    X['nature_cb']=X['nature_cb'].replace(np.nan,'SANS_CB')
    
    X['Parrain']=X['firstnamesponsor'].map(lambda x : 0 if x == np.nan else 1 )
    del X['firstnamesponsor']
    
    X['pays']=X['pays'].replace(np.nan,'NR')
    X['pays_naissance_iso']=X['pays_naissance_iso'].replace(np.nan,'NR')

    nb=-2
    X['regime_matrimonial']=X['regime_matrimonial'].fillna(nb)
    X['patrimoine']=X['patrimoine'].fillna(nb)
    X['revenus_annuels']=X['revenus_annuels'].fillna(nb)
    
    X.rename(columns={'LABEL':cible},inplace=True)

    
    #*************************** III- transformation de donnees ******************
        
    #3.1. transformation de coxbox 
    columns1 = set(X.columns)
    columns1.remove(cible)
    import scipy.stats as sc
    for col in columns1:
        if os.path.exists(cheminmodel+col+'_param.csv') == True:
            param=float(pd.read_csv(col+'_param.csv',encoding='utf-8',sep=';')['valeur'].values)
            paramprod=float(X[col].min())
            if param>=0 and paramprod>=0:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x+1))
                else:
                    X[col]=X[col].map(lambda x : ((x+1)**lambda_ -1)/lambda_)
            if param>=0 and paramprod<0:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-paramprod+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-paramprod+1)**lambda_ -1)/lambda_)
            if param<0 and paramprod<param:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-paramprod+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-paramprod+1)**lambda_ -1)/lambda_)
            if param<0 and paramprod>=param:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-param+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-param+1)**lambda_ -1)/lambda_)
        if X[col].dtype == np.dtype('object'):
            regroup=pd.read_csv(col+'.csv',encoding='utf-8',sep=';',names= ['valeur',col,'count'])
            X=X.merge(regroup,on=col, how="left").set_index(X.index)
            del X['count']
            del X[col]
            X.rename(columns={'valeur':col},inplace=True)
    
    
    columns2 = set(X.columns)
    columns2.remove(cible)
    for col in columns2:
        vide=float(pd.read_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')['valeur'].values)
        X[col]=X[col].fillna(vide)
            
    y=X[cible]
    del X[cible]
    par=list(X.columns)
    par.sort()
    X=pd.DataFrame(X,columns=par)
   
    
    import cPickle
    if os.path.exists(cheminmodel+'MODELE.pickle') == True:
        with open("MODELE.pickle",'rb') as f:
            m = cPickle.load(f)
            proba = pd.DataFrame(y)
            P = m.predict_proba(X)[:, 1]
            proba[probabilite]= P
    else:
        with open("M0.pickle",'rb') as f0:
            m0 = cPickle.load(f0)
        with open("M1.pickle",'rb') as f1:
            m1 = cPickle.load(f1)
        with open("M2.pickle",'rb') as f2:
            m2 = cPickle.load(f2)
        with open("M3.pickle",'rb') as f3:
            m3 = cPickle.load(f3)
        with open("M4.pickle",'rb') as f4:
            m4 = cPickle.load(f4)
        with open("M5.pickle",'rb') as f5:
            m5 = cPickle.load(f5)
        with open("M6.pickle",'rb') as f6:
            m6 = cPickle.load(f6)
        
        proba0 = pd.DataFrame(y)
        P= m0.predict_proba(X)[:, 1]
        proba0[probabilite]= P
        proba1 = pd.DataFrame(y)
        P= m1.predict_proba(X)[:, 1]
        proba1[probabilite]= P
        proba2 = pd.DataFrame(y)
        P= m2.predict_proba(X)[:, 1]
        proba2[probabilite]= P
        proba3 = pd.DataFrame(y)
        P= m3.predict_proba(X)[:, 1]
        proba3[probabilite]= P
        proba4 = pd.DataFrame(y)
        P= m4.predict_proba(X)[:, 1]
        proba4[probabilite]= P
        proba5 = pd.DataFrame(y)
        P= m5.predict_proba(X)[:, 1]
        proba5[probabilite]= P
        proba6 = pd.DataFrame(y)
        P= m6.predict_proba(X)[:, 1]
        proba6[probabilite]= P
        proba = pd.DataFrame(y)
        coef2=pd.read_csv("coefmodel.csv",sep=';',encoding='utf-8')['valeur'].values
        coef=list(coef2)
        proba[probabilite]=coef[0]*proba0[probabilite]+coef[1]*proba1[probabilite]+coef[2]*proba2[probabilite]+coef[3]*proba3[probabilite]+coef[4]*proba4[probabilite]+coef[5]*proba5[probabilite]+coef[6]*proba6[probabilite]        
        
        
    return proba
    

####################################### VI- suivi de performance #################################################


def suivi_performance(table,cible,chemin):
    import numpy as np
    import pandas as pd
    import os

    u= cible
    v="proba"
    probabilite="%s_%s" %(u,v)
    
    if os.path.isdir(chemin)==True:
        os.chdir(chemin)
    else:
        os.makedirs(chemin) 
        os.chdir(chemin)


    table.sort_values(by=probabilite,ascending=True)
    decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
    U=pd.DataFrame(decoup)
    U.rename(columns={probabilite:"decile"},inplace=True)
    decile=pd.concat([table,U],axis=1)
    T1=decile[cible].groupby(decile['decile'])
    T2=pd.DataFrame(T1.mean())
    T2.rename(columns={probabilite:"taux_cible"},inplace=True)
    T2=T2.reset_index()
    result=pd.DataFrame((T2[T2['decile']==0][cible]/table[cible].mean()))
    result.to_csv("PERF.csv", sep=';')
    return result

       

In [11]:
y.head()

contactid
5579018784    0
5520343693    0
4933945696    0
5582788533    0
5592764188    1
Name: ENCOURS_6MOIS, dtype: int64

In [12]:
y.describe()

count    27587.000000
mean         0.199587
std          0.399697
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: ENCOURS_6MOIS, dtype: float64

In [13]:
X.count()

AGE                       27587
Parrain                   27587
campagne                  27587
civilite                  27587
code_postal               27587
code_postale_naissance    27587
codeinsee                 27587
contibuableus             27587
csp                       27587
debit_cb                  27587
flag_banque_principale    27587
flag_epargne              27587
nature_cb                 27587
nom                       27587
patrimoine                27587
pays                      27587
pays_naissance_iso        27587
regime_matrimonial        27587
revenus_annuels           27587
dtype: int64

In [17]:
############################################# II- modelisation: construction du modele  ######################################
def modelisationn (X,y,cible,chemin):  
    import numpy as np
    import pandas as pd
    import os
    u= cible
    v="proba"
    probabilite="%s_%s" %(u,v)
    
    #***********************************I-indicateurs de mesure de performence***********************
    
    def mesure(table,cible):
        import numpy as np
        import pandas as pd
        import os

        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[cible].groupby(decile['decile'])
        T2=pd.DataFrame(T1.mean())
        T2.rename(columns={probabilite:"taux_cible"},inplace=True)
        T2=T2.reset_index()
        result=(T2[T2['decile']==0][cible]/table[cible].mean())
        return result

    def proba_reco(table,cible):
        import numpy as np
        import pandas as pd
        import os

        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[probabilite].groupby(decile['decile'])
        T2=pd.DataFrame(T1.min())
        T2.rename(columns={probabilite:"proba_min_decile"},inplace=True)
        T2=T2.reset_index()
        proba=(T2[T2['decile']==0]["proba_min_decile"])
        return proba
    
    #***********************************II-modelisation ***************************************** 
    
    #1.0.echantillionnage
    from sklearn.cross_validation import train_test_split
    X0_train,X0_test,y0_train,y0_test=train_test_split(X,y,random_state=42)
    
    #1.1. modules necessaire
    
    from sklearn import linear_model,decomposition
    from sklearn.pipeline import Pipeline
    from sklearn.feature_selection import RFE,SelectFromModel
    from sklearn import metrics
    from sklearn import  grid_search
    from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier
    from sklearn.calibration import CalibratedClassifierCV
    from sklearn import svm
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    #1.2. modele 1
    try:
        parameters = {'n_estimators':[150,300,350], 'n_jobs':[-1]}
        rfc= RandomForestClassifier()
        m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        m0 = m0.fit(X0_train,y0_train)
        proba0=pd.DataFrame(y0_test)
        P = m0.predict_proba(X0_test)[:, 1]
        proba0[probabilite]= P
        R0=mesure(proba0,cible)
        R0=pd.DataFrame(R0)
        R0.rename(columns={cible:"R0"},inplace=True)
        print(R0)
        
    except ValueError:
        R0=pd.DataFrame([0],index=[9],columns=["R0"])
        print(R0)

    #1.3. modele 2
    try:
        parameters = {'n_estimators':[300,400], 'n_jobs':[-1]}
        rfc= ExtraTreesClassifier()
        m1=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        m1 = m1.fit(X0_train,y0_train)
        proba1=pd.DataFrame(y0_test)
        P = m1.predict_proba(X0_test)[:, 1]
        proba1[probabilite]= P
        R1=mesure(proba1,cible)
        R1=pd.DataFrame(R1)
        R1.rename(columns={cible:"R1"},inplace=True)
        print(R1)
        
    except ValueError:
        R1=pd.DataFrame([0],index=[9],columns=["R1"])
        print(R1)
    
    #1.4. modele 3
    try:
        parameters = {'n_estimators':[150,300,350]}
        rfc= GradientBoostingClassifier(max_depth=7)
        m2=grid_search.GridSearchCV(rfc,parameters,cv=6)
        m2 = m2.fit(X0_train,y0_train)
        P=m2.predict(X0_test)
        proba2=pd.DataFrame(y0_test)
        P = m2.predict_proba(X0_test)[:, 1]
        proba2[probabilite]= P
        R2=mesure(proba2,cible)
        R2=pd.DataFrame(R2)
        R2.rename(columns={cible:"R2"},inplace=True)
        print(R2)
    
        
    except ValueError:
        R2=pd.DataFrame([0],index=[9],columns=["R2"])
        print(R2)
    
    #1.5.modele 4
    try:
        rfc= RandomForestClassifier(n_estimators=200, n_jobs=-1)
        logistic = linear_model.LogisticRegression(solver='liblinear',C=100,n_jobs=-1)
        select=SelectFromModel(logistic,threshold="median")
        a0 = CalibratedClassifierCV(rfc, cv=6, method='isotonic')
        m3 = Pipeline(steps=[('feature_selection', select),('regression', a0)])
        m3 = m3.fit(X0_train,y0_train)
        P = m3.predict_proba(X0_test)[:, 1]
        proba3=pd.DataFrame(y0_test)
        proba3[probabilite]= P
        R3=mesure(proba3,cible)
        R3=pd.DataFrame(R3)
        R3.rename(columns={cible:"R3"},inplace=True)
        print(R3)
    
    except ValueError:
        R3=pd.DataFrame([0],index=[9],columns=["R3"])
        print(R3)
        
    #1.6.modele 5
    try:
        rfc= GradientBoostingClassifier(n_estimators=200,max_depth=8)
        logistic = linear_model.LogisticRegression(solver='liblinear',C=100,n_jobs=-1)
        select=SelectFromModel(logistic,threshold="median")
        a0 = CalibratedClassifierCV(rfc, cv=6, method='sigmoid')
        m4 = Pipeline(steps=[('feature_selection', select),('regression', a0)])
        m4 = m4.fit(X0_train,y0_train)
        P = m4.predict_proba(X0_test)[:, 1]
        proba4=pd.DataFrame(y0_test)
        proba4[probabilite]= P
        R4=mesure(proba4,cible)
        R4=pd.DataFrame(R4)
        R4.rename(columns={cible:"R4"},inplace=True)
        print(R4)
    
    except ValueError:
        R4=pd.DataFrame([0],index=[9],columns=["R4"])
        print(R4)
    
    #1.7.modele 6
    try:
        etc= ExtraTreesClassifier(n_estimators=400,max_depth=10,n_jobs=-1)
        m5=etc.fit(X0_train,y0_train)
        P = m5.predict_proba(X0_test)[:, 1]
        proba5=pd.DataFrame(y0_test)
        proba5[probabilite]= P
        R5=mesure(proba5,cible)
        R5=pd.DataFrame(R5)
        R5.rename(columns={cible:"R5"},inplace=True)
        print(R5)
    
    except ValueError:
        R5=pd.DataFrame([0],index=[9],columns=["R5"])
        print(R5)
    
    #1.8.modele 7
    try:
        parameters = {'n_estimators':[200,300], 'max_depth':[10,18],'n_jobs':[-1]}
        rfc= RandomForestClassifier()
        m6=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1).fit(X0_train,y0_train)
        proba6=pd.DataFrame(y0_test)
        P = m6.predict_proba(X0_test)[:, 1]
        proba6[probabilite]= P
        R6=mesure(proba6,cible)
        R6=pd.DataFrame(R6)
        R6.rename(columns={cible:"R6"},inplace=True)
        print(R6)
        
    except ValueError:
        R6=pd.DataFrame([0],index=[9],columns=["R6"])
        print(R6)
       
    try:
        coef=[0.14,0.14,0.14,0.14,0.14,0.14,0.14]
        proba7=coef[0]*proba0+coef[1]*proba1+coef[2]*proba2+coef[3]*proba3+coef[4]*proba4+coef[5]*proba5+coef[6]*proba6
        R7=mesure(proba7,cible)
        R7=pd.DataFrame(R7)
        R7.rename(columns={cible:"R7"},inplace=True)
        coef2=coef
        coef=pd.DataFrame(coef2,columns=['valeur'])
        coef.to_csv("coefmodel.csv",sep=';',encoding='utf-8')
        print(R7)
    except ValueError:
        R7=pd.DataFrame([0],index=[9],columns=["R7"])
        print(R7)
    
    try:
        R9=R7
        R9.rename(columns={"R7":"R8"},inplace=True)
        
        for i in xrange(1,100000):
            li=list(np.random.random(7))
            li=li/sum(li)
            coef=list(li)
            proba8=coef[0]*proba0+coef[1]*proba1+coef[2]*proba2+coef[3]*proba3+coef[4]*proba4+coef[5]*proba5+coef[6]*proba6
            R8=mesure(proba8,cible)
            R8=pd.DataFrame(R8)
            R8.rename(columns={cible:"R8"},inplace=True)
            if float(R8["R8"])>float(R9["R8"]):
                R9=R8
                coef2=coef
        coef=pd.DataFrame(coef2,columns=['valeur'])
        coef.to_csv("coefmodel.csv",sep=';',encoding='utf-8')
        print(R9)
    except ValueError:
        R8=pd.DataFrame([0],index=[9],columns=["R8"])
        print(R8)
        
     
    #***********************************III-choix du modele *****************************************
    import cPickle
    plus0="/"
    plus1=cible
    plus2="MOD"
    chemin1=chemin+plus2+plus0+plus1+plus0
    
    if os.path.isdir(chemin1)==True:
        os.chdir(chemin1)
    else:
        os.makedirs(chemin1) 
        os.chdir(chemin1)

        
    R=pd.concat([R0,R1,R2,R3,R4,R5,R6,R7],axis=1)
    RR=R.T
    RR=pd.DataFrame(RR)
    RR.rename(columns=lambda x: 'valeur',inplace=True)
    RR.sort_values(by='valeur',ascending=False,inplace=True)
    RRR=pd.DataFrame(RR.head(1))
    param=RRR.index

     
    if param=="R0":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m0,f)
        export=proba_reco(proba0,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')   
    if param=="R1":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m1,f)
        export=proba_reco(proba1,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    if param=="R2":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m2,f)
        export=proba_reco(proba2,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
        
    if param=="R3":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m3,f)
        export=proba_reco(proba3,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
        
    if param=="R4":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m4,f)
        export=proba_reco(proba4,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R5":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m5,f)
        export=proba_reco(proba5,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R6":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m6,f)
        export=proba_reco(proba6,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R7":
        with open(chemin1+"M0.pickle",'wb') as f:
            cPickle.dump(m0,f)
        with open(chemin1+"M1.pickle",'wb') as f:
            cPickle.dump(m1,f)  
        with open(chemin1+"M2.pickle",'wb') as f:
            cPickle.dump(m2,f)
        with open(chemin1+"M3.pickle",'wb') as f:
            cPickle.dump(m3,f)
        with open(chemin1+"M4.pickle",'wb') as f:
            cPickle.dump(m4,f)
        with open(chemin1+"M5.pickle",'wb') as f:
            cPickle.dump(m5,f)
        with open(chemin1+"M6.pickle",'wb') as f:
            cPickle.dump(m6,f)
        export=proba_reco(proba7,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
        
    
    print(param)

    
####################################### III-Application modele ######################################################

def application(X,cible,cheminmodel):
    
    
    #*************************** I- chargement des donnees pour datamanagement ******************
    import os
    import cPickle
    import numpy as np
    import pandas as pd
    
    u= cible
    v="proba"
    probabilite="%s_%s" %(u,v)
    
    
    if os.path.isdir(cheminmodel)==True:
        os.chdir(cheminmodel)
    else:
        os.makedirs(cheminmodel)
        os.chdir(cheminmodel)
    

 
    var=pd.read_csv("VARIABLE.csv",sep=';',encoding="latin-1")['level_1'].values
    var2=list(var)
    X=pd.DataFrame(X, columns=var2)
    X.rename(columns={'LABEL':cible},inplace=True)

    
    
    #*************************** II- retaitement manuel de variables ******************
    
     
    from datetime import datetime
    X['civilite']=X['civilite'].replace(np.nan,'NR')
    X['code_postal']=X['code_postal'].replace('Vide','-2')
    X['code_postal']=X['code_postal'].replace(np.nan,'-2')


    now=datetime.now()
    X['AGE']=X['date_naissance'].replace(np.nan,'01/01/1800')
    X['AGE']= pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE']=X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']
    
    X['debit_cb']=X['debit_cb'].replace(np.nan,'SANS_CB')
    X['nature_cb']=X['nature_cb'].replace(np.nan,'SANS_CB')
    
    X['Parrain']=X['firstnamesponsor'].map(lambda x : 0 if x == np.nan else 1 )
    del X['firstnamesponsor']
    
    X['pays']=X['pays'].replace(np.nan,'NR')
    X['pays_naissance_iso']=X['pays_naissance_iso'].replace(np.nan,'NR')

    nb=-2
    X['regime_matrimonial']=X['regime_matrimonial'].fillna(nb)
    X['patrimoine']=X['patrimoine'].fillna(nb)
    X['revenus_annuels']=X['revenus_annuels'].fillna(nb)
    
    X.rename(columns={'LABEL':cible},inplace=True)

    
    #*************************** III- transformation de donnees ******************
        
    #3.1. transformation de coxbox 
    columns1 = set(X.columns)
    columns1.remove(cible)
    import scipy.stats as sc
    for col in columns1:
        if os.path.exists(cheminmodel+col+'_param.csv') == True:
            param=float(pd.read_csv(col+'_param.csv',encoding='utf-8',sep=';')['valeur'].values)
            paramprod=float(X[col].min())
            if param>=0 and paramprod>=0:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x+1))
                else:
                    X[col]=X[col].map(lambda x : ((x+1)**lambda_ -1)/lambda_)
            if param>=0 and paramprod<0:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-paramprod+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-paramprod+1)**lambda_ -1)/lambda_)
            if param<0 and paramprod<param:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-paramprod+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-paramprod+1)**lambda_ -1)/lambda_)
            if param<0 and paramprod>=param:
                lambda_=float(pd.read_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')['valeur'].values)
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-param+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-param+1)**lambda_ -1)/lambda_)
        if X[col].dtype == np.dtype('object'):
            regroup=pd.read_csv(col+'.csv',encoding='utf-8',sep=';',names= ['valeur',col,'count'])
            X=X.merge(regroup,on=col, how="left").set_index(X.index)
            del X['count']
            del X[col]
            X.rename(columns={'valeur':col},inplace=True)
    
    
    columns2 = set(X.columns)
    columns2.remove(cible)
    for col in columns2:
        vide=float(pd.read_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')['valeur'].values)
        X[col]=X[col].fillna(vide)
            
    y=X[cible]
    del X[cible]
    par=list(X.columns)
    par.sort()
    X=pd.DataFrame(X,columns=par)
   
    
    import cPickle
    if os.path.exists(cheminmodel+'MODELE.pickle') == True:
        with open("MODELE.pickle",'rb') as f:
            m = cPickle.load(f)
            proba = pd.DataFrame(y)
            P = m.predict_proba(X)[:, 1]
            proba[probabilite]= P
    else:
        with open("M0.pickle",'rb') as f0:
            m0 = cPickle.load(f0)
        with open("M1.pickle",'rb') as f1:
            m1 = cPickle.load(f1)
        with open("M2.pickle",'rb') as f2:
            m2 = cPickle.load(f2)
        with open("M3.pickle",'rb') as f3:
            m3 = cPickle.load(f3)
        with open("M4.pickle",'rb') as f4:
            m4 = cPickle.load(f4)
        with open("M5.pickle",'rb') as f5:
            m5 = cPickle.load(f5)
        with open("M6.pickle",'rb') as f6:
            m6 = cPickle.load(f6)
        
        proba0 = pd.DataFrame(y)
        P= m0.predict_proba(X)[:, 1]
        proba0[probabilite]= P
        proba1 = pd.DataFrame(y)
        P= m1.predict_proba(X)[:, 1]
        proba1[probabilite]= P
        proba2 = pd.DataFrame(y)
        P= m2.predict_proba(X)[:, 1]
        proba2[probabilite]= P
        proba3 = pd.DataFrame(y)
        P= m3.predict_proba(X)[:, 1]
        proba3[probabilite]= P
        proba4 = pd.DataFrame(y)
        P= m4.predict_proba(X)[:, 1]
        proba4[probabilite]= P
        proba5 = pd.DataFrame(y)
        P= m5.predict_proba(X)[:, 1]
        proba5[probabilite]= P
        proba6 = pd.DataFrame(y)
        P= m6.predict_proba(X)[:, 1]
        proba6[probabilite]= P
        proba = pd.DataFrame(y)
        coef2=pd.read_csv("coefmodel.csv",sep=';',encoding='utf-8')['valeur'].values
        coef=list(coef2)
        proba[probabilite]=coef[0]*proba0[probabilite]+coef[1]*proba1[probabilite]+coef[2]*proba2[probabilite]+coef[3]*proba3[probabilite]+coef[4]*proba4[probabilite]+coef[5]*proba5[probabilite]+coef[6]*proba6[probabilite]        
        
        
    return proba
    

####################################### VI- suivi de performance #################################################


def suivi_performance(table,cible,chemin):
    import numpy as np
    import pandas as pd
    import os

    u= cible
    v="proba"
    probabilite="%s_%s" %(u,v)
    
    if os.path.isdir(chemin)==True:
        os.chdir(chemin)
    else:
        os.makedirs(chemin) 
        os.chdir(chemin)


    table.sort_values(by=probabilite,ascending=True)
    decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
    U=pd.DataFrame(decoup)
    U.rename(columns={probabilite:"decile"},inplace=True)
    decile=pd.concat([table,U],axis=1)
    T1=decile[cible].groupby(decile['decile'])
    T2=pd.DataFrame(T1.mean())
    T2.rename(columns={probabilite:"taux_cible"},inplace=True)
    T2=T2.reset_index()
    result=pd.DataFrame((T2[T2['decile']==0][cible]/table[cible].mean()))
    result.to_csv("PERF.csv", sep=';')
    return result

       

In [18]:
modelisationn(X,y,'ENCOURS_6MOIS','/home/foueslat/dtl-pnb/Pilou_workspace/MOD/ENCOURS_6MOIS/')

   R0
9   0
   R1
9   0
   R2
9   0
   R3
9   0
   R4
9   0
   R5
9   0
   R6
9   0


UnboundLocalError: local variable 'proba0' referenced before assignment

In [16]:
y.describe()

count    27587.000000
mean         0.199587
std          0.399697
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: ENCOURS_6MOIS, dtype: float64

In [161]:
def mesure(table,cible):
        import numpy as np
        import pandas as pd
        import os

        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[cible].groupby(decile['decile'])
        T2=pd.DataFrame(T1.mean())
        T2.rename(columns={probabilite:"taux_cible"},inplace=True)
        T2=T2.reset_index()
        result=(T2[T2['decile']==0][cible]/table[cible].mean())
        return result

In [157]:
from sklearn.cross_validation import train_test_split
X0_train,X0_test,y0_train,y0_test=train_test_split(X,y,random_state=42)

In [158]:
X0_train.head()

AGE  Parrain    campagne  civilite  code_postal  \
contactid                                                           
5527847089  25.345040        1  392.570535         0         3152   
5371582816  17.270844        1  392.570535         2         3495   
5507657386  15.420738        1  392.570535         2         2919   
5683387038  21.112190        1  392.570535         0         2259   
5574519612  32.549316        1  325.299757         2         3686   

            code_postale_naissance  codeinsee  contibuableus  csp  debit_cb  \
contactid                                                                     
5527847089                       1       4950      41.797413    6         2   
5371582816                       1       5403      41.797413    4         1   
5507657386                       1       4494      41.797413    2         1   
5683387038                       1       3449      41.797413   18         1   
5574519612                       1       5663      41.797413    6         2   

            flag_banque_principale  flag_epargne  nature_cb   nom  patrimoine  \
contactid                                                                       
5527847089                       0             1          2  6763           1   
5371582816                       0             1          3    73           1   
5507657386                       0             1          1  6126           0   
5683387038                       0             1          3  1537           0   
5574519612                       0             2          2  7138           6   

            pays  pays_naissance_iso  regime_matrimonial  revenus_annuels  
contactid                                                                  
5527847089     1                   1                   2                1  
5371582816     1                   1                   2                1  
5507657386     1                   1                   0                2  
5683387038     1                   1                   2                0  
5574519612     1                   1                   2                4

In [159]:
from sklearn import linear_model,decomposition
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn import metrics
from sklearn import  grid_search
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#1.2. modele 1
try:
    parameters = {'n_estimators':[150,300,350], 'n_jobs':[-1]}
    rfc= RandomForestClassifier()
    m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1).fit(X0_train,y0_train)
    proba0=pd.DataFrame(y0_test)
    P = m0.predict_proba(X0_test)[:, 1]
    proba0["probabilite"]= P
    R0=mesure(proba0,cible)
    R0=pd.DataFrame(R0)
    R0.rename(columns={cible:"R0"},inplace=True)
    print(R0)

except ValueError:
    R0=pd.DataFrame([0],index=[9],columns=["R0"])
    print(R0)


NameError: name 'cible' is not defined

In [ ]:
X0_train.describe()


In [49]:
for col in X0_train.columns:
    print col, sum(pd.isnull(X0_train[col]))

AGE 0
Parrain 0
campagne 0
civilite 0
code_postal 0
code_postale_naissance 0
codeinsee 0
contibuableus 0
csp 0
debit_cb 0
flag_banque_principale 0
flag_epargne 0
nature_cb 0
nom 0
patrimoine 0
pays 0
pays_naissance_iso 0
regime_matrimonial 0
revenus_annuels 0


In [ ]:
m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1).fit(X0_train,y0_train)